In [1]:
from prophet import Prophet
import os
import pandas as pd
import pickle

# Find files to read from
a = os.listdir()
use_list = []
use_df = pd.DataFrame(columns = ["ds","yhat","ride"])
skip_list = []

for el in a:
    if el.endswith(".csv"):
        use_list.append(el)

for ell in use_list:
    
    try:
        load_flag = 0
        c = os.listdir()
        
        for el in c:
            if el == f"{ell}-model.pkl":
                print("FOUND MODEL")
                load_flag += 1
        
        if load_flag == 0:
            
            print(f"\t{ell} - did not find model")
            file = ell
            df = pd.read_csv(file)
            #print(df.columns)
            df.columns = ["date","ds","UNUSED","y"]
            df = df[~df["date"].str.contains('2020') == True]
            df = df[["ds","y"]]
            #print(df.shape)
            df = df.dropna()
            df = df[200000:]
            #print(df.shape)

            #print(df.columns)
            m = Prophet();
            m.fit(df);
            pickle.dump(m, open(f'{ell}-model.pkl', 'wb'))
            future = m.make_future_dataframe(periods=(365));
            forecast = m.predict(future);
            #print(forecast.tail())
            #print("======")
            #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30))
            #print("======")
            future_date = pd.DataFrame({'ds':['2022-11-15 8:00:00',
                                              '2022-11-15 9:00:00',
                                              '2022-11-15 10:00:00',
                                              '2022-11-15 11:00:00',
                                              '2022-11-15 12:00:00',
                                              '2022-11-15 13:00:00',
                                              '2022-11-15 14:00:00',
                                              '2022-11-15 15:00:00',
                                              '2022-11-15 16:00:00',
                                              '2022-11-15 17:00:00',
                                              '2022-11-15 18:00:00',
                                              '2022-11-15 19:00:00',
                                              '2022-11-15 20:00:00',
                                              '2022-11-15 21:00:00',
                                              '2022-11-15 22:00:00',]})
            forecast = m.predict(future_date)
            #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
            temp = {
                "ds": forecast['ds'],
                "yhat": forecast["yhat"],
                "ride": ell
            }
            df_temp = pd.DataFrame(temp)
            use_df = pd.concat([use_df,df_temp])
            print(f"\t\tResulting df shape after {ell}: {use_df.shape}")
        else:
            print(f"found model for {ell}!")
            m = pickle.load(open(f"{ell}-model.pkl", 'rb'))
            future = m.make_future_dataframe(periods=(365));
            forecast = m.predict(future);
            #print(forecast.tail())
            #print("======")
            #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30))
            #print("======")
            future_date = pd.DataFrame({'ds':['2022-11-15 8:00:00',
                                              '2022-11-15 9:00:00',
                                              '2022-11-15 10:00:00',
                                              '2022-11-15 11:00:00',
                                              '2022-11-15 12:00:00',
                                              '2022-11-15 13:00:00',
                                              '2022-11-15 14:00:00',
                                              '2022-11-15 15:00:00',
                                              '2022-11-15 16:00:00',
                                              '2022-11-15 17:00:00',
                                              '2022-11-15 18:00:00',
                                              '2022-11-15 19:00:00',
                                              '2022-11-15 20:00:00',
                                              '2022-11-15 21:00:00',
                                              '2022-11-15 22:00:00',]})
            forecast = m.predict(future_date)
            #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
            temp = {
                "ds": forecast['ds'],
                "yhat": forecast["yhat"],
                "ride": ell
            }
            df_temp = pd.DataFrame(temp)
            use_df = pd.concat([use_df,df_temp])
            print(f"\t\tResulting df shape after {ell}: {use_df.shape}")
            
            
            
    except:
        skip_list.append(ell)
        print(f"\t\tSkipping {ell} due to error")
        
    



# Set up Variables and lists
use_df1 = use_df[use_df["yhat"] > 0]
hours = list(use_df1.ds.unique())
exclude_list = []
result_df = pd.DataFrame(columns = ["ds","yhat","ride"])

# Main loop to build list
for el in hours:
    # Find ride with least time, and that has not been excluded
    a = use_df1[use_df1["ds"] == el]
    a = a[~a["ride"].isin(exclude_list)]
    a = a.sort_values(by = "yhat")
    b = {
        "ds": [a.iloc[0][0]],
        'yhat': [a.iloc[0][1]],
        'ride': [a.iloc[0][2]]
    }
    
    # Exclude ride from further selections!
    exclude_list.append(a.iloc[0][2])
    
    # Concat dataframes to build result df
    bdf = pd.DataFrame(b)
    result_df = pd.concat([result_df,bdf])

result_df

FOUND MODEL
1
found model for 7_dwarfs_train.csv!
		Resulting df shape after 7_dwarfs_train.csv: (15, 3)
0
	ariel_s_grotto.csv - did not find model
		Skipping ariel_s_grotto.csv due to error
FOUND MODEL
1
found model for astro_orbiter.csv!
		Resulting df shape after astro_orbiter.csv: (30, 3)
FOUND MODEL
1
found model for barnstormer.csv!
		Resulting df shape after barnstormer.csv: (45, 3)
FOUND MODEL
1
found model for big_thunder_mtn.csv!
		Resulting df shape after big_thunder_mtn.csv: (60, 3)
FOUND MODEL
1
found model for buzz_lightyear.csv!
		Resulting df shape after buzz_lightyear.csv: (75, 3)
0
	carousel_of_progress.csv - did not find model
		Skipping carousel_of_progress.csv due to error
0
	casey_jr_splash.csv - did not find model
		Skipping casey_jr_splash.csv due to error
0
	country_bears.csv - did not find model
		Skipping country_bears.csv due to error
FOUND MODEL
1
found model for dumbo.csv!
		Resulting df shape after dumbo.csv: (90, 3)
0
	enchanted_tiki_rm.csv - did not fin

,ds,yhat,ride
0,2022-11-15 08:00:00,2.516061,7_dwarfs_train.csv
0,2022-11-15 09:00:00,1.691172,pirates_of_caribbean.csv
0,2022-11-15 10:00:00,1.624477,philharmagic.csv
0,2022-11-15 11:00:00,11.944992,town_sq_mickey.csv
0,2022-11-15 12:00:00,67.902071,mad_tea_party.csv
0,2022-11-15 13:00:00,70.265664,buzz_lightyear.csv
0,2022-11-15 14:00:00,73.174926,winnie_the_pooh.csv
0,2022-11-15 15:00:00,77.592580,astro_orbiter.csv
0,2022-11-15 16:00:00,93.927535,magic_carpets.csv
0,2022-11-15 17:00:00,94.348298,haunted_mansion.csv
